# Laborator 3

In [1]:
import requests
import mwparserfromhell

import numpy as np

import re

## Parserul de text (Exercitiul 1)

In [2]:
response = requests.get(
    'https://ro.wikipedia.org/w/api.php',
    params={
        'action': 'query',
        'format': 'json',
        'titles': 'Astronomie',
        'prop': 'revisions',
        'rvprop': 'content'
    }).json()
page = next(iter(response['query']['pages'].values()))
wikicode = page['revisions'][0]['*']
parsed_wikicode = mwparserfromhell.parse(wikicode)
print(parsed_wikicode.strip_code()[:1500])

Acest articol este despre studiul științific al obiectelor cerești.
Nu trebuie confundat cu pseudoștiința astrologie.
[[File:Crab Nebula.jpg|thumb|Nebuloasa Crabului, o rămășiță de supernovă. Foto: Hubble]]
[[File:The Galactic Centre and Bulge above the ESO 3.6-metre telescope.jpg|thumb|Calea Lactee văzută de la Observatorul La Silla, Chile]]
Astronomia ( / ástronomía) este o știință naturală care studiază obiecte și fenomene cerești. Folosește matematica, fizica și chimia pentru a explica originea și evoluția lor. Printre obiectele de interes se numără: planete, sateliți naturali, stele, nebuloase, galaxii și comete. Fenomenele relevante includ explozii de supernove, explozii de raze gamma, quasari, blazari, pulsari, și radiații cosmice de fond. Mai general, astronomia studiază tot ceea ce își are originea în afara atmosferei Pământului. Cosmologia este o ramură a astronomiei și studiază Universul în ansamblu.

Astronomia este una dintre cele mai vechi științe naturale. Civilizațiile 

In [3]:
corpus_data = parsed_wikicode.strip_code()
len(corpus_data.replace('\n',' ').split(' '))

6893

In [4]:
len(set(corpus_data.replace('\n',' ').split(' ')))

2816

## Modelul de limba n-gram (Exercitiul 2)

### Preprocesare

In [5]:
test_corpus='There is a big house. I buy a house. They buy the new house.'

In [6]:
def preprocess_corpus(corpus_data: str):
    result_list = []
    preprocessed_text = corpus_data.split('\n')
    for paragraph in preprocessed_text:
        sentence = re.split('[.?!]\s*', paragraph.lower())
        for el in sentence:
            if el != '' and not el.startswith('[[file:') and not el.endswith(']]') and not 'thumb|' in el:
                el = ' <s> ' + el + ' </s> '
                result_list.append(el)
    return result_list

In [7]:
preprocess_corpus(test_corpus)

[' <s> there is a big house </s> ',
 ' <s> i buy a house </s> ',
 ' <s> they buy the new house </s> ']

In [8]:
preprocess_corpus(corpus_data)

[' <s> acest articol este despre studiul științific al obiectelor cerești </s> ',
 ' <s> nu trebuie confundat cu pseudoștiința astrologie </s> ',
 ' <s> 6-metre telescope </s> ',
 ' <s> astronomia ( / ástronomía) este o știință naturală care studiază obiecte și fenomene cerești </s> ',
 ' <s> folosește matematica, fizica și chimia pentru a explica originea și evoluția lor </s> ',
 ' <s> printre obiectele de interes se numără: planete, sateliți naturali, stele, nebuloase, galaxii și comete </s> ',
 ' <s> fenomenele relevante includ explozii de supernove, explozii de raze gamma, quasari, blazari, pulsari, și radiații cosmice de fond </s> ',
 ' <s> mai general, astronomia studiază tot ceea ce își are originea în afara atmosferei pământului </s> ',
 ' <s> cosmologia este o ramură a astronomiei și studiază universul în ansamblu </s> ',
 ' <s> astronomia este una dintre cele mai vechi științe naturale </s> ',
 ' <s> civilizațiile timpurii din istoria înregistrată au făcut observații metodice

### N-grams

In [9]:
def compute_ngrams(n_grams: int, corpus: list, smoothing=True):
    n_gram_probabilities = dict()
    unique_words = set()
    for sentence in corpus:
        sentence_list = sentence.split(' ')
        for index_word in range (0, len(sentence_list)):
            word = sentence_list[index_word]
            if word not in ['', '<s>', '</s>'] :
                nominator = '. '.join(corpus).count(' ' + ' '.join(sentence_list[index_word-n_grams + 1:index_word]) + ' ' + word + ' ')
                denominator = '. '.join(corpus).count(' ' + ' '.join(sentence_list[index_word-n_grams + 1:index_word]) + ' ')
                unique_words.add(word)
                n_gram_probabilities[(word, ' '.join(sentence_list[index_word-n_grams + 1:index_word]))] = (nominator, denominator, nominator/denominator)
    if smoothing:
        for key,value in n_gram_probabilities.items():
            # print(key, value)
            n_gram_probabilities[key] = (value[0] + 1, value[1] + len(unique_words), value[0] / (value[1] + len(unique_words))) # Laplace - Add one rule
    return n_gram_probabilities, unique_words

### Bigram test

In [10]:
language_model, words = compute_ngrams(2,preprocess_corpus(test_corpus))
print('Modelul de limbaj bigram pentru tot corpusul de la seminar este {0}'.format(language_model))
print('In total sunt {0} cuvinte unice in corpusul de la seminar.'.format(len(words)))
print('Ele sunt: {0}.'.format(words))

Modelul de limbaj bigram pentru tot corpusul de la seminar este {('there', '<s>'): (2, 13, 0.07692307692307693), ('is', 'there'): (2, 11, 0.09090909090909091), ('a', 'is'): (2, 11, 0.09090909090909091), ('big', 'a'): (2, 12, 0.08333333333333333), ('house', 'big'): (2, 11, 0.09090909090909091), ('i', '<s>'): (2, 13, 0.07692307692307693), ('buy', 'i'): (2, 11, 0.09090909090909091), ('a', 'buy'): (2, 12, 0.08333333333333333), ('house', 'a'): (2, 12, 0.08333333333333333), ('they', '<s>'): (2, 13, 0.07692307692307693), ('buy', 'they'): (2, 11, 0.09090909090909091), ('the', 'buy'): (2, 12, 0.08333333333333333), ('new', 'the'): (2, 11, 0.09090909090909091), ('house', 'new'): (2, 11, 0.09090909090909091)}
In total sunt 10 cuvinte unice in corpusul de la seminar.
Ele sunt: {'big', 'they', 'house', 'buy', 'new', 'a', 'is', 'there', 'i', 'the'}.


In [11]:
language_model, words = compute_ngrams(2,preprocess_corpus(corpus_data))
print('Modelul de limbaj bigram pentru tot corpusul de pe wikipedia este {0}'.format(language_model))

Modelul de limbaj bigram pentru tot corpusul de pe wikipedia este {('acest', '<s>'): (3, 2830, 0.0007067137809187279), ('articol', 'acest'): (2, 2419, 0.00041339396444811904), ('este', 'articol'): (2, 2412, 0.00041459369817578774), ('despre', 'este'): (2, 2490, 0.00040160642570281126), ('studiul', 'despre'): (2, 2422, 0.00041288191577208916), ('științific', 'studiul'): (2, 2424, 0.00041254125412541255), ('al', 'științific'): (2, 2413, 0.0004144218814753419), ('obiectelor', 'al'): (2, 2430, 0.00041152263374485596), ('cerești', 'obiectelor'): (5, 2424, 0.0016501650165016502), ('nu', '<s>'): (2, 2830, 0.00035335689045936394), ('trebuie', 'nu'): (3, 2425, 0.0008247422680412372), ('confundat', 'trebuie'): (2, 2415, 0.00041407867494824016), ('cu', 'confundat'): (2, 2412, 0.00041459369817578774), ('pseudoștiința', 'cu'): (2, 2472, 0.0004045307443365696), ('astrologie', 'pseudoștiința'): (2, 2413, 0.0004144218814753419), ('6-metre', '<s>'): (2, 2830, 0.00035335689045936394), ('telescope', '6-m

In [12]:
print('In total sunt {0} cuvinte unice in corpusul de pe wikipedia.'.format(len(words)))

In total sunt 2411 cuvinte unice in corpusul de pe wikipedia.


### Trigram test

In [13]:
language_model, words = compute_ngrams(3,preprocess_corpus(test_corpus))
print('Modelul de limbaj trigram pentru tot corpusul de la seminar este {0}'.format(language_model))
print('In total sunt {0} cuvinte unice in corpusul de la seminar.'.format(len(words)))
print('Ele sunt: {0}.'.format(words))

Modelul de limbaj trigram pentru tot corpusul de la seminar este {('there', ' <s>'): (1, 12, 0.0), ('is', '<s> there'): (2, 11, 0.09090909090909091), ('a', 'there is'): (2, 11, 0.09090909090909091), ('big', 'is a'): (2, 11, 0.09090909090909091), ('house', 'a big'): (2, 11, 0.09090909090909091), ('i', ' <s>'): (2, 12, 0.08333333333333333), ('buy', '<s> i'): (2, 11, 0.09090909090909091), ('a', 'i buy'): (2, 11, 0.09090909090909091), ('house', 'buy a'): (2, 11, 0.09090909090909091), ('they', ' <s>'): (2, 12, 0.08333333333333333), ('buy', '<s> they'): (2, 11, 0.09090909090909091), ('the', 'they buy'): (2, 11, 0.09090909090909091), ('new', 'buy the'): (2, 11, 0.09090909090909091), ('house', 'the new'): (2, 11, 0.09090909090909091)}
In total sunt 10 cuvinte unice in corpusul de la seminar.
Ele sunt: {'big', 'they', 'house', 'buy', 'new', 'a', 'is', 'there', 'i', 'the'}.


In [14]:
language_model, words = compute_ngrams(3,preprocess_corpus(corpus_data))
print('Modelul de limbaj trigram pentru tot corpusul de pe wikipedia este {0}'.format(language_model))

Modelul de limbaj trigram pentru tot corpusul de pe wikipedia este {('acest', ' <s>'): (2, 2829, 0.0003534817956875221), ('articol', '<s> acest'): (2, 2413, 0.0004144218814753419), ('este', 'acest articol'): (2, 2412, 0.00041459369817578774), ('despre', 'articol este'): (2, 2412, 0.00041459369817578774), ('studiul', 'este despre'): (2, 2412, 0.00041459369817578774), ('științific', 'despre studiul'): (2, 2412, 0.00041459369817578774), ('al', 'studiul științific'): (2, 2412, 0.00041459369817578774), ('obiectelor', 'științific al'): (2, 2412, 0.00041459369817578774), ('cerești', 'al obiectelor'): (2, 2412, 0.00041459369817578774), ('nu', ' <s>'): (2, 2829, 0.0003534817956875221), ('trebuie', '<s> nu'): (2, 2412, 0.00041459369817578774), ('confundat', 'nu trebuie'): (2, 2413, 0.0004144218814753419), ('cu', 'trebuie confundat'): (2, 2412, 0.00041459369817578774), ('pseudoștiința', 'confundat cu'): (2, 2412, 0.00041459369817578774), ('astrologie', 'cu pseudoștiința'): (2, 2412, 0.00041459369

## Predictia unei noi propozitii (Exercitiul 3)

In [15]:
new_sentence = 'They buy a red house.'
preprocess_corpus(new_sentence)

[' <s> they buy a red house </s> ']

In [16]:
def predict_for_new_sentence(n_grams: int, training_data: list, new_sentence_list: list):
    used_model_ngram = dict()
    training_data = preprocess_corpus(training_data)
    model, unique_words = compute_ngrams(n_grams,training_data)
    for sentence in new_sentence_list:
        list_of_words = sentence.split(' ')
        for index_word in range(0, len(list_of_words)):
            word = list_of_words[index_word]
            if word not in ['', '<s>', '</s>']:
                if (word, ' '.join(list_of_words[index_word-n_grams + 1:index_word])) in model:
                    used_model_ngram[(word, ' '.join(list_of_words[index_word-n_grams + 1:index_word]))] = model[(word, ' '.join(list_of_words[index_word-n_grams + 1:index_word]))]
                else:
                    nominator = 1
                    denominator = len(unique_words)
                    for key in model.keys():
                        if key[1] == ' '.join(list_of_words[index_word-n_grams + 1:index_word]):
                            denominator = model[key][1]
                            break
                    used_model_ngram[(word, ' '.join(list_of_words[index_word-n_grams + 1:index_word]))] = (nominator, denominator, nominator/denominator)
    probability = 1
    for values in used_model_ngram.values():
        probability *= values[2]
    return used_model_ngram, probability

### Test pe ce am facut la laborator

In [17]:
model, probability = predict_for_new_sentence(2, test_corpus, preprocess_corpus(new_sentence))
print('Modelul de limba bigram folosit pentru exemplul de la seminar este: \n{0}'.format(model))
print('Probabilitatea de aparitie a propozitiei de test THEY BUY A RED HOUSE. este {0} %.'.format(probability * 100))

Modelul de limba bigram folosit pentru exemplul de la seminar este: 
{('they', '<s>'): (2, 13, 0.07692307692307693), ('buy', 'they'): (2, 11, 0.09090909090909091), ('a', 'buy'): (2, 12, 0.08333333333333333), ('red', 'a'): (1, 12, 0.08333333333333333), ('house', 'red'): (1, 10, 0.1)}
Probabilitatea de aparitie a propozitiei de test THEY BUY A RED HOUSE. este 0.0004856254856254856 %.


### Test pe corpusul din wikipedia

In [18]:
new_sentence_1 = 'Ana are mere.'
new_sentence_2 = 'Hubble este un telescop spațial.'

model1, probability1 = predict_for_new_sentence(2, corpus_data, preprocess_corpus(new_sentence_1))
model2, probability2 = predict_for_new_sentence(2, corpus_data, preprocess_corpus(new_sentence_2))
print('Modelul de limba bigram folosit pentru prima propozitie este: \n{0}'.format(model1))
print('Probabilitate pentru prima propozitie este {0} %.'.format(probability1 * 100))
print('\n')
print('Modelul de limba bigram folosit pentru a doua propozitie este: \n{0}'.format(model2))
print('Probabilitate pentru a doua propozitie este {0} %.'.format(probability2 * 100))

Modelul de limba bigram folosit pentru prima propozitie este: 
{('ana', '<s>'): (1, 2830, 0.00035335689045936394), ('are', 'ana'): (1, 2411, 0.00041476565740356696), ('mere', 'are'): (1, 2417, 0.0004137360364087712)}
Probabilitate pentru prima propozitie este 6.0637278845452336e-09 %.


Modelul de limba bigram folosit pentru a doua propozitie este: 
{('hubble', '<s>'): (1, 2830, 0.00035335689045936394), ('este', 'hubble'): (1, 2413, 0.0004144218814753419), ('un', 'este'): (6, 2490, 0.002008032128514056), ('telescop', 'un'): (1, 2443, 0.00040933278755628325), ('spațial', 'telescop'): (1, 2411, 0.00041476565740356696)}
Probabilitate pentru a doua propozitie este 4.992363765845127e-15 %.
